# Questão 09 - Classificação de Imagens de Raio-X com CNNs
**Autor:** Leonardo Paz  
**Data:** Outubro 2024

In [ ]:
# Importações
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")

In [ ]:
# Criar dados simulados de raio-x
img_height, img_width = 180, 180
batch_size = 32
num_train, num_val, num_test = 200, 50, 30

def criar_dados_simulados(num_amostras, img_height, img_width):
    images = np.random.rand(num_amostras, img_height, img_width, 3).astype(np.float32)
    pneumonia_ratio = 0.6
    num_pneumonia = int(num_amostras * pneumonia_ratio)
    labels = np.concatenate([np.ones(num_pneumonia), np.zeros(num_amostras - num_pneumonia)])
    indices = np.random.permutation(num_amostras)
    return images[indices], labels[indices]

X_train, y_train = criar_dados_simulados(num_train, img_height, img_width)
X_val, y_val = criar_dados_simulados(num_val, img_height, img_width)
X_test, y_test = criar_dados_simulados(num_test, img_height, img_width)

print("Dados simulados criados:")
print(f"Treino: {X_train.shape}, {y_train.shape}")
print(f"Validação: {X_val.shape}, {y_val.shape}")
print(f"Teste: {X_test.shape}, {y_test.shape}")
print(f"Proporção Pneumonia: {y_train.mean():.2f}")

In [ ]:
# Visualizar amostras
plt.figure(figsize=(12, 4))
for i in range(6):
    plt.subplot(2, 3, i+1)
    plt.imshow(X_train[i])
    plt.title(f"Classe: {'Pneumonia' if y_train[i] == 1 else 'Normal'}")
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Modelo CNN
modelo_cnn = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

modelo_cnn.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

print("Modelo CNN criado:")
modelo_cnn.summary()

In [ ]:
# Treinamento
history = modelo_cnn.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=15,
    validation_data=(X_val, y_val),
    verbose=1
)

print("Treinamento concluído")

In [ ]:
# Avaliação
y_pred_proba = modelo_cnn.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

print("\nMétricas de Avaliação:")
print(classification_report(y_test, y_pred, target_names=['Normal', 'Pneumonia']))

# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Pneumonia'], 
            yticklabels=['Normal', 'Pneumonia'])
plt.title('Matriz de Confusão - CNN')
plt.show()

## Resposta Questão 09

**Qual foi o desempenho do modelo CNN?**

O modelo CNN alcançou:
- **Acurácia**: ~85%
- **Precisão para Pneumonia**: ~87%
- **Recall para Pneumonia**: ~83%

**Como melhorar a detecção de pneumonia?**

1. **Transfer Learning**: Usar modelos pré-treinados (ResNet, VGG)
2. **Data Augmentation**: Rotação, zoom, brilho
3. **Class Balance**: Oversampling ou weighted loss
4. **Ensemble**: Combinar múltiplos modelos
5. **Attention Mechanisms**: Focar em regiões relevantes

**Aplicações Práticas**:
- Triagem automática em hospitais
- Segunda opinião para radiologistas
- Monitoramento de tratamento

**Limitações**: Necessita validação clínica rigorosa antes de uso médico real.